# Model Experimentation: Hyperparameter Optimization
This notebook conducts experiments on hyperparameter optimization.

## Imports and Setup

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import sys
sys.path.insert(0, '../utils')
import model_utils
import geoutils

import logging
import warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

SEED = 42

%load_ext autoreload
%autoreload 2

## File Location

In [2]:
data_dir = "../data/"
input_file = data_dir + '20200409_dataset.csv'

pos_mask_dir = data_dir + 'pos_masks/'
neg_mask_dir = data_dir + 'neg_masks/'
sentinel_dir = data_dir + 'sentinel2/'
false_pos_dir = data_dir + 'false_pos/'

areas = ['maicao', 'riohacha', 'uribia']

## Load Dataset

In [3]:
data = pd.read_csv(input_file).reset_index(drop=True)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

Data dimensions: (690163, 112)


,B1_2016,B2_2016,B3_2016,B4_2016,B5_2016,B6_2016,B7_2016,B8_2016,B9_2016,B10_2016,...,savi_2020,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,target,area
0,0.15970,0.13735,0.1531,0.18700,0.2090,0.26320,0.30515,0.26965,0.3327,0.0411,...,0.187614,-0.509745,0.105128,0.239614,0.449106,-0.718433,-0.042537,0.901237,3,0
1,0.15970,0.13905,0.1454,0.17845,0.2090,0.26320,0.30515,0.26395,0.3327,0.0411,...,0.177058,-0.507485,0.105128,0.247826,0.464498,-0.716955,-0.039760,0.911490,3,0
2,0.16675,0.14875,0.1589,0.18605,0.2258,0.27945,0.32070,0.28085,0.3452,0.0416,...,0.179191,-0.524371,0.073259,0.262348,0.446475,-0.722188,-0.033995,0.875915,3,0


## Resample Dataset
Resamples 30,000 negative examples per area.

In [4]:
la_guajira = data['area'].apply(lambda x: model_utils.get_la_guajira(x)) 
data.insert(len(data.columns)-2, 'la_guajira', la_guajira)

data = model_utils.resample(data, num_neg_samples=30000, false_pos_samples=1000, random_state=SEED)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

Data dimensions: (169647, 113)


,B1_2016,B2_2016,B3_2016,B4_2016,B5_2016,B6_2016,B7_2016,B8_2016,B9_2016,B10_2016,...,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,la_guajira,target,area
0,0.1809,0.17790,0.1972,0.2371,0.2427,0.3009,0.33460,0.2913,0.3655,0.04435,...,-0.388713,0.040390,0.386829,0.574649,-0.603260,-0.010778,0.810930,1,4,0
1,0.1684,0.13690,0.1425,0.1458,0.1686,0.2696,0.30675,0.2652,0.3501,0.03810,...,-0.602441,0.098756,0.199285,0.318486,-0.752879,-0.057824,0.809911,1,4,0
2,0.1635,0.13655,0.1327,0.1346,0.1311,0.1754,0.20490,0.2100,0.2202,0.03560,...,-0.506141,0.128252,0.180842,0.403433,-0.754468,-0.048452,0.950226,1,4,0


## Machine Learning Pipeline

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from lightgbm import LGBMClassifier
SEED = 42

## Define Features and Target Label

In [7]:
# Get target and feature columns
label = 'target'
features = [column for column in data.columns[:-2]]

# Convert target labels to binary
data[label] = data[label].replace({2:0, 3:0, 4:0})

print('Data dimensions: {}'.format(data.shape))
print('Class distribution:\n{}'.format(data['target'].value_counts()))

Data dimensions: (169647, 113)
Class distribution:
0    150000
1     19647
Name: target, dtype: int64


## Logistic Regression

### Feature Selection: Recursive Feature Eleimination CV

In [8]:
lr = LogisticRegression()
lr_best_features = model_utils.rfecv_feature_selection(
    lr, data, features, label, scoring='f1', step=10, verbose=0
)
print(lr_best_features)

['B2_2016', 'B3_2016', 'B5_2016', 'B6_2016', 'B7_2016', 'B8_2016', 'B9_2016', 'B10_2016', 'B12_2016', 'ndvi_2016', 'ndbi_2016', 'savi_2016', 'mndwi_2016', 'ui_2016', 'nbi_2016', 'brba_2016', 'nbai_2016', 'mbi_2016', 'B1_2017', 'B2_2017', 'B3_2017', 'B4_2017', 'B5_2017', 'B6_2017', 'B8_2017', 'B10_2017', 'B11_2017', 'B12_2017', 'mndwi_2017', 'ui_2017', 'nbi_2017', 'brba_2017', 'nbai_2017', 'mbi_2017', 'baei_2017', 'B2_2018', 'B5_2018', 'B6_2018', 'B7_2018', 'B8_2018', 'B9_2018', 'B10_2018', 'B12_2018', 'ndbi_2018', 'savi_2018', 'ui_2018', 'nbi_2018', 'mbi_2018', 'baei_2018', 'B1_2019', 'B2_2019', 'B4_2019', 'B5_2019', 'B6_2019', 'B7_2019', 'B8_2019', 'B9_2019', 'B10_2019', 'B11_2019', 'B12_2019', 'ndvi_2019', 'ndbi_2019', 'savi_2019', 'mndwi_2019', 'ui_2019', 'nbi_2019', 'brba_2019', 'nbai_2019', 'mbi_2019', 'baei_2019', 'B1_2020', 'B2_2020', 'B3_2020', 'B4_2020', 'B5_2020', 'B6_2020', 'B7_2020', 'B8_2020', 'B9_2020', 'B10_2020', 'B11_2020', 'B12_2020', 'ndvi_2020', 'savi_2020', 'mndwi_

### [Baseline] Logistic Regression Results Sans Hyperparamater Optimization

In [9]:
lr = LogisticRegression()
results, clfs = model_utils.geospatialcv(data, lr_best_features, label, lr, verbose=2);


Test set: MAICAO
[[29358   642]
 [  143   567]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     30000
           1       0.47      0.80      0.59       710

    accuracy                           0.97     30710
   macro avg       0.73      0.89      0.79     30710
weighted avg       0.98      0.97      0.98     30710

MAICAO Results: 
- F1 Score: 0.5909
- Kappa Statistics: 0.5787
- Precision: 0.4690
- Recall: 0.7986
- Accuracy: 0.9744

Test set: RIOHACHA
[[29477   523]
 [ 1452  2049]]
              precision    recall  f1-score   support

           0       0.95      0.98      0.97     30000
           1       0.80      0.59      0.67      3501

    accuracy                           0.94     33501
   macro avg       0.87      0.78      0.82     33501
weighted avg       0.94      0.94      0.94     33501

RIOHACHA Results: 
- F1 Score: 0.6748
- Kappa Statistics: 0.6432
- Precision: 0.7967
- Recall: 0.5853
- Accuracy: 0.9410

Test 

### Grid Search Cross Validation

In [17]:
param_grid = {
    'classifier__penalty': ['l2'],
    'classifier__C':[0.001, 0.01, 1.0, 5.0, 10]
}
lr_grid_search = model_utils.hyperparameter_optimization(
    data, lr_best_features, label, lr, param_grid, scoring='f1', verbose=2
)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:   25.1s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   30.8s finished


Best Paramaters: {'classifier__C': 1.0, 'classifier__penalty': 'l2'}


### Logistic Regression Results with Hyperparameter Optimization

In [19]:
lr = LogisticRegression(C=1.0)
results, clfs = model_utils.geospatialcv(data, lr_best_features, label, lr, verbose=2);


Test set: MAICAO
[[29358   642]
 [  143   567]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     30000
           1       0.47      0.80      0.59       710

    accuracy                           0.97     30710
   macro avg       0.73      0.89      0.79     30710
weighted avg       0.98      0.97      0.98     30710

MAICAO Results: 
- F1 Score: 0.5909
- Kappa Statistics: 0.5787
- Precision: 0.4690
- Recall: 0.7986
- Accuracy: 0.9744

Test set: RIOHACHA
[[29477   523]
 [ 1452  2049]]
              precision    recall  f1-score   support

           0       0.95      0.98      0.97     30000
           1       0.80      0.59      0.67      3501

    accuracy                           0.94     33501
   macro avg       0.87      0.78      0.82     33501
weighted avg       0.94      0.94      0.94     33501

RIOHACHA Results: 
- F1 Score: 0.6748
- Kappa Statistics: 0.6432
- Precision: 0.7967
- Recall: 0.5853
- Accuracy: 0.9410

Test 

## LinearSVC

In [20]:
lsvc = LinearSVC(random_state=SEED)
lsvc_best_features = model_utils.rfecv_feature_selection(
    lr, data, features, label, scoring='f1', step=10, verbose=0
)
print(lsvc_best_features)

['B2_2016', 'B3_2016', 'B5_2016', 'B6_2016', 'B7_2016', 'B8_2016', 'B9_2016', 'B10_2016', 'B12_2016', 'ndvi_2016', 'ndbi_2016', 'savi_2016', 'mndwi_2016', 'ui_2016', 'nbi_2016', 'brba_2016', 'nbai_2016', 'mbi_2016', 'B1_2017', 'B2_2017', 'B3_2017', 'B4_2017', 'B5_2017', 'B6_2017', 'B8_2017', 'B10_2017', 'B11_2017', 'B12_2017', 'mndwi_2017', 'ui_2017', 'nbi_2017', 'brba_2017', 'nbai_2017', 'mbi_2017', 'baei_2017', 'B2_2018', 'B5_2018', 'B6_2018', 'B7_2018', 'B8_2018', 'B9_2018', 'B10_2018', 'B12_2018', 'ndbi_2018', 'savi_2018', 'ui_2018', 'nbi_2018', 'mbi_2018', 'baei_2018', 'B1_2019', 'B2_2019', 'B4_2019', 'B5_2019', 'B6_2019', 'B7_2019', 'B8_2019', 'B9_2019', 'B10_2019', 'B11_2019', 'B12_2019', 'ndvi_2019', 'ndbi_2019', 'savi_2019', 'mndwi_2019', 'ui_2019', 'nbi_2019', 'brba_2019', 'nbai_2019', 'mbi_2019', 'baei_2019', 'B1_2020', 'B2_2020', 'B3_2020', 'B4_2020', 'B5_2020', 'B6_2020', 'B7_2020', 'B8_2020', 'B9_2020', 'B10_2020', 'B11_2020', 'B12_2020', 'ndvi_2020', 'savi_2020', 'mndwi_

### [Baseline] LinearSVC Results Sans Hyperparamater Optimization

In [22]:
lsvc = LinearSVC(random_state=SEED)
results, clfs = model_utils.geospatialcv(data, lsvc_best_features, label, lsvc, verbose=2);


Test set: MAICAO
[[29293   707]
 [  153   557]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     30000
           1       0.44      0.78      0.56       710

    accuracy                           0.97     30710
   macro avg       0.72      0.88      0.77     30710
weighted avg       0.98      0.97      0.98     30710

MAICAO Results: 
- F1 Score: 0.5643
- Kappa Statistics: 0.5510
- Precision: 0.4407
- Recall: 0.7845
- Accuracy: 0.9720

Test set: RIOHACHA
[[29703   297]
 [ 1324  2177]]
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     30000
           1       0.88      0.62      0.73      3501

    accuracy                           0.95     33501
   macro avg       0.92      0.81      0.85     33501
weighted avg       0.95      0.95      0.95     33501

RIOHACHA Results: 
- F1 Score: 0.7287
- Kappa Statistics: 0.7030
- Precision: 0.8800
- Recall: 0.6218
- Accuracy: 0.9516

Test 

### Grid Search Cross Validation
Reference: https://towardsdatascience.com/svm-hyper-parameter-tuning-using-gridsearchcv-49c0bc55ce29

In [23]:
lsvc = LinearSVC(random_state=SEED)
param_grid = {'classifier__C': [0.001, 0.1, 1, 10, 100, 1000]}
lsvc_grid_search = model_utils.hyperparameter_optimization(
    data, lsvc_best_features, label, lsvc, param_grid, scoring='f1', verbose=2
)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.9min finished


Best Paramaters: {'classifier__C': 1000}


### LinearSVC Results with Hyperparameter Optimization

In [24]:
lsvc = LinearSVC(C=1000, random_state=SEED)
results, clfs = model_utils.geospatialcv(data, lsvc_best_features, label, lsvc, verbose=2);


Test set: MAICAO
[[29391   609]
 [  200   510]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     30000
           1       0.46      0.72      0.56       710

    accuracy                           0.97     30710
   macro avg       0.72      0.85      0.77     30710
weighted avg       0.98      0.97      0.98     30710

MAICAO Results: 
- F1 Score: 0.5577
- Kappa Statistics: 0.5448
- Precision: 0.4558
- Recall: 0.7183
- Accuracy: 0.9737

Test set: RIOHACHA
[[29630   370]
 [ 1059  2442]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     30000
           1       0.87      0.70      0.77      3501

    accuracy                           0.96     33501
   macro avg       0.92      0.84      0.88     33501
weighted avg       0.96      0.96      0.96     33501

RIOHACHA Results: 
- F1 Score: 0.7736
- Kappa Statistics: 0.7504
- Precision: 0.8684
- Recall: 0.6975
- Accuracy: 0.9573

Test 